In [ ]:
import pandas as pd
import numpy as np

In [ ]:
transactions = pd.read_excel("E-commerce_data.xlsx",sheet_name="transactions")
customers = pd.read_excel("E-commerce_data.xlsx",sheet_name="customers")

In [ ]:
customers

In [ ]:
transactions.head()

In [ ]:
transactions.tail()

In [ ]:
transactions.shape

In [ ]:
transactions.dtypes

In [ ]:
transactions.describe()

In [ ]:
transactions.isna().sum()

In [ ]:
(transactions.isna().sum() * 100)/transactions.shape[0]

In [ ]:
#add the city to the dataset
transactions_test = transactions
transactions_test["city_id"] = " "

In [ ]:
#fill city column
i = 0
for value in transactions_test["customer_id"]:
    transactions_test["city_id"][i] = customers["city_id"][value-1]
    i = i+1
transactions_test

In [ ]:
# encodel the transaction_status column burned = 1 , subscribed = 0
i = 0
for value in transactions_test["transaction_status"]:
    if(value == "burned"):
        transactions_test["transaction_status"][i] = 1
    else:
        transactions_test["transaction_status"][i] = 0
    i = i+1
transactions_test



In [ ]:
#add gender column
transactions_test["gender_id"] = 0
transactions_test

In [ ]:
#fill gender column
i = 0
for value in transactions_test["customer_id"]:
    transactions_test["gender_id"][i] = customers["gender_id"][value-1]
    i = i+1
transactions_test

In [ ]:
#create coupon_usage in customers table
customers["coupon_usage"] = 0
customers

In [ ]:
#add coupon usage to customer table 
ind = transactions_test["customer_id"].value_counts().index
v = (transactions_test["customer_id"].value_counts()).values
i = 0
for value in ind:
    customers["coupon_usage"][value-1] = v[i]
    i= i+1
customers

In [ ]:
#add and fill coupon usage to transactions table
transactions_test["coupon_usage"] = 0
i = 0
for value in transactions_test["customer_id"] :
    transactions_test["coupon_usage"][i] = customers["coupon_usage"][value -1] 
    i = i+1
transactions_test

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#feature selection & find the correlation
transactions_corr = transactions_test.drop(columns=["customer_id","transaction_date","transaction_id","coupon_name","burn_date"]) 
transactions_corr.corr()

In [ ]:
#check if there are outliers 
for col in transactions_corr.columns:
    q1 = transactions_corr[col].quantile(0.25)
    q3 = transactions_corr[col].quantile(0.75)
    IQR = q3- q1
    lower_limit = q1 - 1.5*IQR
    upper_limit = q3 + 1.5*IQR
    print("the outliers for ", col)
    print("the upper_limit = ",upper_limit,"the lower_limit = ",lower_limit)
    print("\n")

In [ ]:
transactions_corr.dtypes

In [ ]:
transactions_corr["transaction_status"] = transactions_corr.transaction_status.astype(str).astype(int)
transactions_corr["city_id"] = transactions_corr["city_id"].astype(str).astype(int)

In [ ]:
transactions_corr.dtypes

The outliers are very low and they are in the coupon usage column so we don't need to remove them

In [ ]:
sns.relplot(x = "transaction_status", y = "coupon_usage",hue="gender_id",data=transactions_corr[:5000])

In [ ]:
sns.relplot(x = "city_id", y = "branch_id",hue="coupon_usage",data=transactions_corr[:5000])

In [ ]:
sns.relplot(x = "coupon_usage", y = "gender_id",hue="transaction_status",data=transactions_corr[:5000])

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [ ]:
X = transactions_corr
X

In [ ]:
# Elbow method.
X_k = transactions_corr.iloc[:, [0,4]]# transaction status with coupon usage
Z_k = transactions_corr.iloc[:,[2,4]] # city id with coupon usage 
A_k = transactions_corr.iloc[:,[3,4] ] # gender id , coupon usage
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_k)
Z_scaled = scaler.fit_transform(Z_k)
A_scaled= scaler.fit_transform(A_k)
wsse = []
for k in range(1,10):    #X_k
    kmeans = KMeans(n_clusters=k,random_state=40)
    kmeans.fit(X_scaled)
    wsse.append(kmeans.inertia_)
wsse2 = []
for k1 in range(1,10):    #Y_k
    kmeans2 = KMeans(n_clusters=k1,random_state=40)
    kmeans2.fit(X_scaled)
    wsse2.append(kmeans2.inertia_)

wsse3 = []
for k3 in range(1,10):    #Z_k
    kmeans3 = KMeans(n_clusters=k3,random_state=40)
    kmeans3.fit(X_scaled)
    wsse3.append(kmeans3.inertia_)

plt.figure(figsize=(12, 6))

plt.subplot(223)
plt.plot(range(1,10),wsse,marker = "x")
plt.title('elbow method')
plt.xlabel("Number of clusters")
plt.ylabel("wsse")

plt.subplot(211)
plt.plot(range(1,10),wsse2,marker = "x")
plt.title('elbow method')
plt.xlabel("Number of clusters")
plt.ylabel("wsse2")
plt.subplot(224)
plt.plot(range(1,10),wsse3,marker = "x")
plt.title('elbow method')
plt.ylabel("wsse3")
plt.xlabel("Number of clusters")

plt.tight_layout()
plt.show()

so the Kmean = 4

In [ ]:
# Train the model
kmeans = KMeans(n_clusters=4,random_state=42)
kmeans.fit(X_k)
X_pred = kmeans.predict(X_k)
transactions_corr["cluster_x"] = kmeans.labels_

kmeans2 = KMeans(n_clusters=4,random_state=42)
kmeans2.fit(Z_k)
Z_pred = kmeans2.predict(Z_k)
transactions_corr["cluster_z"] = kmeans2.labels_


kmeans3 = KMeans(n_clusters=4,random_state=42)
kmeans3.fit(A_k)
A_pred = kmeans3.predict(A_k)
transactions_corr["cluster_a"] = kmeans3.labels_

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(223)
plt.scatter(X_k["transaction_status"],X_k["coupon_usage"], c=transactions_corr['cluster_x'], cmap='viridis')
plt.title('K-means Clustering')
plt.xlabel('transaction_status')
plt.ylabel('coupon_usage')

plt.subplot(211)
plt.scatter(Z_k['city_id'], Z_k['coupon_usage'], c=transactions_corr['cluster_z'], cmap='viridis')
plt.title('K-means Clustering')
plt.xlabel('city_id')
plt.ylabel('coupon_usage')
plt.subplot(224)
plt.scatter(A_k['gender_id'], A_k['coupon_usage'], c=transactions_corr['cluster_a'], cmap='viridis')
plt.title('K-means Clustering')
plt.ylabel('coupon_usage')
plt.xlabel('gender_id')

plt.tight_layout()
plt.show()

In [ ]:
# evaluate the model
from sklearn.metrics import silhouette_score
print(silhouette_score(X_scaled,X_pred))
print(silhouette_score(Z_scaled,Z_pred))
print(silhouette_score(A_scaled,A_pred))


# explain the Kmeans cluster here.
Based on silhouette score we can see that : the model is not good and that because of the data and the k-means model is not the best model for this data.

so based on the k-means :
in the K-means clustering X: transaction status Y: coupon usage we have 4 clusters 
blue: they are the most loyalty customers and we must to get them more coupons because they have attention to the products so we must to keep it.
purple: they are the second loyalty customers and if they get more coupons, there are big probability to become loyalty customers.
green: they don't have big attention to the products.
yellow: I suggest to stop giving them.
The same thing applies on the another clustering X:gender id, Y: coupon usage.
# Note for gender id {1: Male , 2:Female}.
# Note2 : as we said the model is not good so we can't depend on it.

///////////////////////////////////////////

# DBSCAN:

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

# Create feature matrix
Y = transactions_corr.iloc[:, [0,4]]# transaction status with coupon usage
Z = transactions_corr.iloc[:,[2,4]] # city id with coupon usage 
A = transactions_corr.iloc[:,[3,4] ] # gender id , coupon usage

# Standardize the features
scaler = StandardScaler()
Y_scaled = scaler.fit_transform(Y)
Z_scaled = scaler.fit_transform(Z)
A_scaled= scaler.fit_transform(A)



# DBSCAN clustering
dbscan = DBSCAN(eps=0.5, min_samples=5)
Y_clusters = dbscan.fit_predict(Y_scaled)
Z_clusters = dbscan.fit_predict(Z_scaled)
A_clusters = dbscan.fit_predict(A_scaled)



In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(223)
plt.scatter(Y["transaction_status"],Y['coupon_usage'], c=Y_clusters, cmap='viridis',s=50)
plt.title('DBSCAN')
plt.xlabel('transaction_status')
plt.ylabel('coupon_usage')

plt.subplot(211)
plt.scatter(Z['city_id'], Z['coupon_usage'], c=Z_clusters, cmap='viridis',s=50)
plt.title('DBSCAN')
plt.xlabel('city_id')
plt.ylabel('coupon_usage')
plt.subplot(224)
plt.scatter(A['gender_id'], A['coupon_usage'], c=A_clusters, cmap='viridis',s=50)
plt.title('DBSCAN')
plt.xlabel('gender_id')
plt.ylabel('coupon_usage')

plt.tight_layout()
plt.show()

In [ ]:
from sklearn import metrics
print(metrics.silhouette_score(Y_scaled, Y_clusters))
print(metrics.silhouette_score(Z_scaled, Z_clusters))
print(metrics.silhouette_score(A_scaled, A_clusters))

# DBSCAN analysis and explaining :
Based on silhouette_score we can see : 
the clustering based on transaction status and coupon usage , gender id and coupon usage is better than city id and  coupon usage .
so:
the clustering based on transaction status and coupon usage :
we have 2 clusters
purple : the people who take coupons but didn't use it yet I recommend to stop get them more coupons ,we can get them more coupons when transaction status be burned in this way we can know they will become customers .
yellow : the people who already loyalty customers we must to give them more coupons to keep them royalty.

the seconde clustering that based on the gender and coupon usage note: 1 : male , 2 : female
we can see that there are 3 clusters :
blue : it is the male customers.
purple : it is the female clusters.
yellow : it is also female cluster but they use more than 12 coupon.

the clustering based on the cities and coupon usage :
we can see 4 clusters 
purple: it is the most common cluster 
green : it is only one point and it is the most loyalty customers in their city.
yellow : it is only one point and it is the most loyalty customers in their city.
blue: it is only one point and it is the most loyalty customers in their city.



# That was every thing thank you.
# ＜（＾－＾）＞